# Regression analysis and training

In [1]:
import glob
import os
import os.path
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import logging
from sklearn.preprocessing import StandardScaler
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.annotations import Title
from maweight import model_selection
from config import elastix_params
import pickle
output_notebook()

from config import thigh_features_path, breast_features_path, xls_path, path_prefix_results

import warnings
warnings.filterwarnings('ignore')

# setting the logging format
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

Executables being used: /opt/elastix-5.1.0-linux/bin/elastix /opt/elastix-5.1.0-linux/bin/transformix


Loading BokehJS ...

## Constructing the training dataframe

In [2]:
# Reading the extracted features
thigh_features= pd.read_csv(thigh_features_path)
breast_features= pd.read_csv(breast_features_path)

# Determining the ids and positions
thigh_features['id']= thigh_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4]) 
breast_features['id']= breast_features['filename'].apply(lambda x: x.split(os.sep)[-1]).apply(lambda x: x[:-7] if x.endswith(".nii.gz") else x[:-4])

# Reading the XLS data
xls_data= pd.read_excel(xls_path, engine='openpyxl')

# Extracting the ground truth data
target= pd.DataFrame(data= {'ct_num': xls_data[u'CT_num'].astype(str).str.zfill(3),
                               'pos': xls_data[u'position'].astype(str),
                               'thigh': xls_data[u'thigh'].astype(float),
                               'breast': xls_data[u'breast'].astype(float)})

target['id']= target['ct_num'] + target['pos']
target= target.sort_values('id')
target= target.reset_index()

# Merging the thigh features with the ground truth data
thigh_target= target[['id', 'thigh', 'pos']]
breast_target= target[['id', 'breast', 'pos']]

thigh_data= pd.merge(thigh_features, thigh_target, how='inner', on='id').dropna()
breast_data= pd.merge(breast_features, breast_target, how='inner', on='id').dropna()

In [3]:
thigh_target= thigh_data['thigh']
thigh_features= thigh_data.drop(['filename', 'id', 'thigh'], axis='columns')
breast_target= breast_data['breast']
breast_features= breast_data.drop(['filename', 'id', 'breast'], axis='columns')

# encoding the positions
thigh_features['pos']= thigh_features['pos'].apply(lambda x: {'a': 0, 'k': 1, 'f': 2}[x])
breast_features['pos']= breast_features['pos'].apply(lambda x: {'a': 0, 'k': 1, 'f': 2}[x])

## Model selection with feature selection

In [4]:
masks= np.unique([c.split('-')[1] for c in thigh_features.columns if ('mean_mask' not in c and len(c) > 10) and len(c.split('-')[1]) == 4])

In [5]:
masks

array(['074k', '075a', '075k', '077k', '079k', '080k', '082k', '083a',
       '083k', '084a', '087f', '088f', '088k', '090a', '091f', '092k'],
      dtype='<U4')

In [6]:
results= []
results_no_fs= []

thigh_target.to_csv("/datac3/scripts/github/chicken_ct_weights_test2/chicken_ct_weights/data/tigh_target.csv", index=False)

### thigh using all features

In [7]:
results.append(model_selection(thigh_features, thigh_target, dataset='thigh', type='all'))

Objective KNNR_Objective:


 27%|██████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                       | 2180/8000 [05:15<14:01,  6.92it/s]


iterations: 2180
Number of used features: 40
Used features: ['hist-7-074k-thigh.nii.gz-0.500000', 'mean-075k-thigh.nii.gz-0.500000', 'std-075k-thigh.nii.gz-0.500000', 'num-077k-thigh.nii.gz-0.500000', 'std-077k-thigh.nii.gz-0.500000', 'hist-0-077k-thigh.nii.gz-0.500000', 'hist-1-077k-thigh.nii.gz-0.500000', 'hist-15-077k-thigh.nii.gz-0.500000', 'hist-1-079k-thigh.nii.gz-0.500000', 'num-080k-thigh.nii.gz-0.500000', 'mean-080k-thigh.nii.gz-0.500000', 'hist-1-080k-thigh.nii.gz-0.500000', 'hist-3-080k-thigh.nii.gz-0.500000', 'hist-15-080k-thigh.nii.gz-0.500000', 'sum-082k-thigh.nii.gz-0.500000', 'mean-082k-thigh.nii.gz-0.500000', 'kurt-083a-thigh.nii.gz-0.500000', 'hist-1-083a-thigh.nii.gz-0.500000', 'hist-6-083a-thigh.nii.gz-0.500000', 'sum-083k-thigh.nii.gz-0.500000', 'kurt-083k-thigh.nii.gz-0.500000', 'hist-3-083k-thigh.nii.gz-0.500000', 'hist-5-083k-thigh.nii.gz-0.500000', 'hist-9-083k-thigh.nii.gz-0.500000', 'hist-13-083k-thigh.nii.gz-0.500000', 'hist-14-083k-thigh.nii.gz-0.500000', '

200it [00:00, 683.05it/s]


1
1 0.6547964948403343
Objective LinearRegression_Objective:


...

1
1 0.9647355880469479
Objective PLSRegression_Objective:


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8000/8000 [25:53<00:00,  5.15it/s]


iterations: 7999
Number of used features: 48
Used features: ['num-074k-thigh.nii.gz-0.500000', 'hist-18-074k-thigh.nii.gz-0.500000', 'num-075a-thigh.nii.gz-0.500000', 'hist-4-075a-thigh.nii.gz-0.500000', 'hist-13-075a-thigh.nii.gz-0.500000', 'hist-15-075a-thigh.nii.gz-0.500000', 'hist-18-075a-thigh.nii.gz-0.500000', 'hist-1-075k-thigh.nii.gz-0.500000', 'hist-11-075k-thigh.nii.gz-0.500000', 'hist-15-077k-thigh.nii.gz-0.500000', 'hist-4-079k-thigh.nii.gz-0.500000', 'hist-13-079k-thigh.nii.gz-0.500000', 'num-080k-thigh.nii.gz-0.500000', 'hist-11-080k-thigh.nii.gz-0.500000', 'hist-13-080k-thigh.nii.gz-0.500000', 'hist-14-080k-thigh.nii.gz-0.500000', 'hist-15-080k-thigh.nii.gz-0.500000', 'hist-19-080k-thigh.nii.gz-0.500000', 'hist-0-082k-thigh.nii.gz-0.500000', 'hist-15-082k-thigh.nii.gz-0.500000', 'hist-16-082k-thigh.nii.gz-0.500000', 'hist-19-082k-thigh.nii.gz-0.500000', 'kurt-083a-thigh.nii.gz-0.500000', 'hist-4-083a-thigh.nii.gz-0.500000', 'hist-15-083a-thigh.nii.gz-0.500000', 'hist-16-

200it [00:00, 507.38it/s]

1
1 0.9004386427057365


### thigh using the mean mask features

In [8]:
results.append(model_selection(thigh_features[[c for c in thigh_features.columns if 'mean_mask' in c or c == 'type']], thigh_target, dataset='thigh', type='mean_mask'))

Objective KNNR_Objective:


 16%|████████████████████████████████████████████                                                                                                                                                                                                                                      | 1287/8000 [02:02<10:38, 10.51it/s]


iterations: 1287
Number of used features: 3
Used features: ['hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-5-0.500000-mean_mask'] 
Score: -0.6085143417150146


200it [00:00, 736.62it/s]


1
1 0.6100212718431253
Objective LinearRegression_Objective:


...

1
1 0.6181103675282424
Objective PLSRegression_Objective:


  7%|██████████████████▎                                                                                                                                                                                                                                                                | 532/8000 [00:19<04:34, 27.24it/s]


iterations: 532
Number of used features: 4
Used features: ['hist-4-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-17-0.500000-mean_mask'] 
Score: -0.5875410850941607


200it [00:00, 604.80it/s]

1
1 0.5954952320688666


### thigh using the features of the individual masks

In [9]:
for m in masks:
    results.append(model_selection(thigh_features[[c for c in thigh_features.columns if m in c or c == 'type']], thigh_target, dataset='thigh', type=m))

Objective KNNR_Objective:


 13%|██████████████████████████████████▍                                                                                                                                                                                                                                               | 1005/8000 [01:47<12:28,  9.35it/s]


iterations: 1005
Number of used features: 5
Used features: ['num-074k-thigh.nii.gz-0.500000', 'std-074k-thigh.nii.gz-0.500000', 'kurt-074k-thigh.nii.gz-0.500000', 'hist-0-074k-thigh.nii.gz-0.500000', 'hist-5-074k-thigh.nii.gz-0.500000'] 
Score: -0.6090790340932697


200it [00:00, 734.07it/s]


1
1 0.6210841450295906
Objective LinearRegression_Objective:


...

1
1 0.589730856803278
Objective PLSRegression_Objective:


 10%|██████████████████████████▊                                                                                                                                                                                                                                                        | 779/8000 [00:43<06:41, 17.97it/s]


iterations: 779
Number of used features: 7
Used features: ['num-092k-thigh.nii.gz-0.500000', 'sum-092k-thigh.nii.gz-0.500000', 'kurt-092k-thigh.nii.gz-0.500000', 'hist-3-092k-thigh.nii.gz-0.500000', 'hist-4-092k-thigh.nii.gz-0.500000', 'hist-5-092k-thigh.nii.gz-0.500000', 'hist-6-092k-thigh.nii.gz-0.500000'] 
Score: -0.5876183644228192


200it [00:00, 686.99it/s]

1
1 0.5979262055364727


### breast using all features

In [10]:
results.append(model_selection(breast_features, breast_target, dataset='breast', type='all'))

Objective KNNR_Objective:


 15%|████████████████████████████████████████                                                                                                                                                                                                                                          | 1168/8000 [02:44<16:00,  7.11it/s]


iterations: 1168
Number of used features: 23
Used features: ['hist-15-074k-breast.nii.gz-0.500000', 'kurt-077k-breast.nii.gz-0.500000', 'hist-1-077k-breast.nii.gz-0.500000', 'hist-14-079k-breast.nii.gz-0.500000', 'num-080k-breast.nii.gz-0.500000', 'sum-080k-breast.nii.gz-0.500000', 'hist-6-080k-breast.nii.gz-0.500000', 'hist-11-080k-breast.nii.gz-0.500000', 'num-082k-breast.nii.gz-0.500000', 'hist-1-083a-breast.nii.gz-0.500000', 'hist-6-083k-breast.nii.gz-0.500000', 'sum-088f-breast.nii.gz-0.500000', 'hist-18-088f-breast.nii.gz-0.500000', 'num-088k-breast.nii.gz-0.500000', 'sum-088k-breast.nii.gz-0.500000', 'std-088k-breast.nii.gz-0.500000', 'std-090a-breast.nii.gz-0.500000', 'hist-6-090a-breast.nii.gz-0.500000', 'num-091f-breast.nii.gz-0.500000', 'hist-13-091f-breast.nii.gz-0.500000', 'num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'hist-17-0.500000-mean_mask'] 
Score: -0.8525601507055522


200it [00:00, 708.12it/s]


1
1 0.856548518254646
Objective LinearRegression_Objective:


...

1
1 0.9951324892410475
Objective PLSRegression_Objective:


 46%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                  | 3719/8000 [12:16<14:08,  5.05it/s]


iterations: 3719
Number of used features: 27
Used features: ['sum-075a-breast.nii.gz-0.500000', 'hist-3-075a-breast.nii.gz-0.500000', 'hist-4-075k-breast.nii.gz-0.500000', 'hist-4-077k-breast.nii.gz-0.500000', 'hist-14-079k-breast.nii.gz-0.500000', 'mean-083a-breast.nii.gz-0.500000', 'hist-1-083a-breast.nii.gz-0.500000', 'hist-3-083k-breast.nii.gz-0.500000', 'num-084a-breast.nii.gz-0.500000', 'skew-084a-breast.nii.gz-0.500000', 'hist-19-084a-breast.nii.gz-0.500000', 'hist-1-087f-breast.nii.gz-0.500000', 'hist-2-087f-breast.nii.gz-0.500000', 'mean-088f-breast.nii.gz-0.500000', 'hist-0-088f-breast.nii.gz-0.500000', 'hist-3-088f-breast.nii.gz-0.500000', 'mean-088k-breast.nii.gz-0.500000', 'hist-4-088k-breast.nii.gz-0.500000', 'std-090a-breast.nii.gz-0.500000', 'kurt-090a-breast.nii.gz-0.500000', 'hist-2-090a-breast.nii.gz-0.500000', 'mean-091f-breast.nii.gz-0.500000', 'std-091f-breast.nii.gz-0.500000', 'std-092k-breast.nii.gz-0.500000', 'hist-11-092k-breast.nii.gz-0.500000', 'num-0.500000

200it [00:00, 487.05it/s]

1
1 0.9589731017917894


### breast using the mean mask features

In [11]:
results.append(model_selection(breast_features[[c for c in breast_features.columns if 'mean_mask' in c or c == 'type']], breast_target, dataset='breast', type='mean_mask'))

Objective KNNR_Objective:


  9%|█████████████████████████                                                                                                                                                                                                                                                          | 730/8000 [01:15<12:36,  9.61it/s]


iterations: 730
Number of used features: 4
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-17-0.500000-mean_mask'] 
Score: -0.8507829977737915


200it [00:00, 698.44it/s]


1
1 0.8553703915485695
Objective LinearRegression_Objective:


...

1
1 0.9142884555019826
Objective PLSRegression_Objective:


  9%|█████████████████████████                                                                                                                                                                                                                                                          | 730/8000 [00:46<07:38, 15.86it/s]


iterations: 730
Number of used features: 10
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.9048973696020423


200it [00:00, 636.57it/s]

1
1 0.9061627691585916


### breast using the features of the individual masks

In [12]:
for m in masks:
    results.append(model_selection(breast_features[[c for c in breast_features.columns if m in c or c == 'type']], breast_target, dataset='breast', type=m))

Objective KNNR_Objective:


 14%|██████████████████████████████████████                                                                                                                                                                                                                                            | 1110/8000 [02:07<13:09,  8.73it/s]


iterations: 1110
Number of used features: 4
Used features: ['num-074k-breast.nii.gz-0.500000', 'sum-074k-breast.nii.gz-0.500000', 'hist-6-074k-breast.nii.gz-0.500000', 'hist-12-074k-breast.nii.gz-0.500000'] 
Score: -0.8447759146663291


200it [00:00, 692.31it/s]


1
1 0.8496190342126176
Objective LinearRegression_Objective:


...

1
1 0.9060188594904754
Objective PLSRegression_Objective:


  8%|███████████████████████▎                                                                                                                                                                                                                                                           | 679/8000 [00:32<05:49, 20.97it/s]


iterations: 679
Number of used features: 6
Used features: ['num-092k-breast.nii.gz-0.500000', 'hist-4-092k-breast.nii.gz-0.500000', 'hist-5-092k-breast.nii.gz-0.500000', 'hist-6-092k-breast.nii.gz-0.500000', 'hist-7-092k-breast.nii.gz-0.500000', 'hist-8-092k-breast.nii.gz-0.500000'] 
Score: -0.8977752380851067


200it [00:00, 612.61it/s]

1
1 0.898710980144736


### Saving the results

In [13]:
results= pd.concat(results)
results.to_csv(os.path.join(path_prefix_results,'results.csv'), index=False)
pickle.dump(results, open(os.path.join(path_prefix_results,'results.pickle'), 'wb'))

## Without feature selection

### thigh with all features

In [14]:
results_no_fs.append(model_selection(thigh_features, thigh_target, dataset='thigh', type='all', disable_feature_selection=True))

Objective KNNR_Objective:


  7%|██████████████████▍                                                                                                                                                                                                                                                                | 538/8000 [00:42<09:55, 12.52it/s]


iterations: 538
Number of used features: 443
Used features: ['num-074k-thigh.nii.gz-0.500000', 'sum-074k-thigh.nii.gz-0.500000', 'mean-074k-thigh.nii.gz-0.500000', 'std-074k-thigh.nii.gz-0.500000', 'skew-074k-thigh.nii.gz-0.500000', 'kurt-074k-thigh.nii.gz-0.500000', 'hist-0-074k-thigh.nii.gz-0.500000', 'hist-1-074k-thigh.nii.gz-0.500000', 'hist-2-074k-thigh.nii.gz-0.500000', 'hist-3-074k-thigh.nii.gz-0.500000', 'hist-4-074k-thigh.nii.gz-0.500000', 'hist-5-074k-thigh.nii.gz-0.500000', 'hist-6-074k-thigh.nii.gz-0.500000', 'hist-7-074k-thigh.nii.gz-0.500000', 'hist-8-074k-thigh.nii.gz-0.500000', 'hist-9-074k-thigh.nii.gz-0.500000', 'hist-10-074k-thigh.nii.gz-0.500000', 'hist-11-074k-thigh.nii.gz-0.500000', 'hist-12-074k-thigh.nii.gz-0.500000', 'hist-13-074k-thigh.nii.gz-0.500000', 'hist-14-074k-thigh.nii.gz-0.500000', 'hist-15-074k-thigh.nii.gz-0.500000', 'hist-16-074k-thigh.nii.gz-0.500000', 'hist-17-074k-thigh.nii.gz-0.500000', 'hist-18-074k-thigh.nii.gz-0.500000', 'hist-19-074k-thigh.

200it [00:00, 585.33it/s]


1
1 0.3993651128746345
Objective LinearRegression_Objective:


...

1
1 0.29227633085933025
Objective PLSRegression_Objective:


  8%|████████████████████▋                                                                                                                                                                                                                                                              | 601/8000 [00:11<02:17, 53.92it/s]


iterations: 601
Number of used features: 443
Used features: ['num-074k-thigh.nii.gz-0.500000', 'sum-074k-thigh.nii.gz-0.500000', 'mean-074k-thigh.nii.gz-0.500000', 'std-074k-thigh.nii.gz-0.500000', 'skew-074k-thigh.nii.gz-0.500000', 'kurt-074k-thigh.nii.gz-0.500000', 'hist-0-074k-thigh.nii.gz-0.500000', 'hist-1-074k-thigh.nii.gz-0.500000', 'hist-2-074k-thigh.nii.gz-0.500000', 'hist-3-074k-thigh.nii.gz-0.500000', 'hist-4-074k-thigh.nii.gz-0.500000', 'hist-5-074k-thigh.nii.gz-0.500000', 'hist-6-074k-thigh.nii.gz-0.500000', 'hist-7-074k-thigh.nii.gz-0.500000', 'hist-8-074k-thigh.nii.gz-0.500000', 'hist-9-074k-thigh.nii.gz-0.500000', 'hist-10-074k-thigh.nii.gz-0.500000', 'hist-11-074k-thigh.nii.gz-0.500000', 'hist-12-074k-thigh.nii.gz-0.500000', 'hist-13-074k-thigh.nii.gz-0.500000', 'hist-14-074k-thigh.nii.gz-0.500000', 'hist-15-074k-thigh.nii.gz-0.500000', 'hist-16-074k-thigh.nii.gz-0.500000', 'hist-17-074k-thigh.nii.gz-0.500000', 'hist-18-074k-thigh.nii.gz-0.500000', 'hist-19-074k-thigh.

200it [00:00, 325.85it/s]


1
1 0.46303100575167866


### thigh mean mask

In [15]:
results_no_fs.append(model_selection(thigh_features[[c for c in thigh_features.columns if 'mean_mask' in c or c == 'type']], thigh_target, dataset='thigh', type='mean_mask', disable_feature_selection=True))

Objective KNNR_Objective:


  7%|███████████████████▎                                                                                                                                                                                                                                                               | 563/8000 [00:15<03:27, 35.77it/s]


iterations: 563
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.32545417537347865


200it [00:00, 743.96it/s]


1
1 0.3422546022830222
Objective LinearRegression_Objective:


...

1
1 0.5136800770324552
Objective PLSRegression_Objective:


  6%|█████████████████▏                                                                                                                                                                                                                                                                | 502/8000 [00:00<00:07, 967.03it/s]


iterations: 502
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.4895716537354793


200it [00:00, 653.90it/s]

1
1 0.5111383749775585


### thigh individual masks

In [16]:
for m in masks:
    results_no_fs.append(model_selection(thigh_features[[c for c in thigh_features.columns if m in c or c == 'type']], thigh_target, dataset='thigh', type=m, disable_feature_selection=True))

Objective KNNR_Objective:


  7%|███████████████████▏                                                                                                                                                                                                                                                               | 559/8000 [00:15<03:32, 35.01it/s]


iterations: 559
Number of used features: 26
Used features: ['num-074k-thigh.nii.gz-0.500000', 'sum-074k-thigh.nii.gz-0.500000', 'mean-074k-thigh.nii.gz-0.500000', 'std-074k-thigh.nii.gz-0.500000', 'skew-074k-thigh.nii.gz-0.500000', 'kurt-074k-thigh.nii.gz-0.500000', 'hist-0-074k-thigh.nii.gz-0.500000', 'hist-1-074k-thigh.nii.gz-0.500000', 'hist-2-074k-thigh.nii.gz-0.500000', 'hist-3-074k-thigh.nii.gz-0.500000', 'hist-4-074k-thigh.nii.gz-0.500000', 'hist-5-074k-thigh.nii.gz-0.500000', 'hist-6-074k-thigh.nii.gz-0.500000', 'hist-7-074k-thigh.nii.gz-0.500000', 'hist-8-074k-thigh.nii.gz-0.500000', 'hist-9-074k-thigh.nii.gz-0.500000', 'hist-10-074k-thigh.nii.gz-0.500000', 'hist-11-074k-thigh.nii.gz-0.500000', 'hist-12-074k-thigh.nii.gz-0.500000', 'hist-13-074k-thigh.nii.gz-0.500000', 'hist-14-074k-thigh.nii.gz-0.500000', 'hist-15-074k-thigh.nii.gz-0.500000', 'hist-16-074k-thigh.nii.gz-0.500000', 'hist-17-074k-thigh.nii.gz-0.500000', 'hist-18-074k-thigh.nii.gz-0.500000', 'hist-19-074k-thigh.n

200it [00:00, 765.52it/s]


1
1 0.3719431391838929
Objective LinearRegression_Objective:


...

1
1 0.46726366953104526
Objective PLSRegression_Objective:


  6%|█████████████████▏                                                                                                                                                                                                                                                                | 502/8000 [00:00<00:07, 953.66it/s]


iterations: 502
Number of used features: 26
Used features: ['num-092k-thigh.nii.gz-0.500000', 'sum-092k-thigh.nii.gz-0.500000', 'mean-092k-thigh.nii.gz-0.500000', 'std-092k-thigh.nii.gz-0.500000', 'skew-092k-thigh.nii.gz-0.500000', 'kurt-092k-thigh.nii.gz-0.500000', 'hist-0-092k-thigh.nii.gz-0.500000', 'hist-1-092k-thigh.nii.gz-0.500000', 'hist-2-092k-thigh.nii.gz-0.500000', 'hist-3-092k-thigh.nii.gz-0.500000', 'hist-4-092k-thigh.nii.gz-0.500000', 'hist-5-092k-thigh.nii.gz-0.500000', 'hist-6-092k-thigh.nii.gz-0.500000', 'hist-7-092k-thigh.nii.gz-0.500000', 'hist-8-092k-thigh.nii.gz-0.500000', 'hist-9-092k-thigh.nii.gz-0.500000', 'hist-10-092k-thigh.nii.gz-0.500000', 'hist-11-092k-thigh.nii.gz-0.500000', 'hist-12-092k-thigh.nii.gz-0.500000', 'hist-13-092k-thigh.nii.gz-0.500000', 'hist-14-092k-thigh.nii.gz-0.500000', 'hist-15-092k-thigh.nii.gz-0.500000', 'hist-16-092k-thigh.nii.gz-0.500000', 'hist-17-092k-thigh.nii.gz-0.500000', 'hist-18-092k-thigh.nii.gz-0.500000', 'hist-19-092k-thigh.n

200it [00:00, 641.77it/s]

1
1 0.5060887312294099


### breast all features

In [17]:
results_no_fs.append(model_selection(breast_features, breast_target, dataset='breast', type='all', disable_feature_selection=True))

Objective KNNR_Objective:


  7%|██████████████████▋                                                                                                                                                                                                                                                                | 545/8000 [00:43<09:57, 12.48it/s]


iterations: 545
Number of used features: 443
Used features: ['num-074k-breast.nii.gz-0.500000', 'sum-074k-breast.nii.gz-0.500000', 'mean-074k-breast.nii.gz-0.500000', 'std-074k-breast.nii.gz-0.500000', 'skew-074k-breast.nii.gz-0.500000', 'kurt-074k-breast.nii.gz-0.500000', 'hist-0-074k-breast.nii.gz-0.500000', 'hist-1-074k-breast.nii.gz-0.500000', 'hist-2-074k-breast.nii.gz-0.500000', 'hist-3-074k-breast.nii.gz-0.500000', 'hist-4-074k-breast.nii.gz-0.500000', 'hist-5-074k-breast.nii.gz-0.500000', 'hist-6-074k-breast.nii.gz-0.500000', 'hist-7-074k-breast.nii.gz-0.500000', 'hist-8-074k-breast.nii.gz-0.500000', 'hist-9-074k-breast.nii.gz-0.500000', 'hist-10-074k-breast.nii.gz-0.500000', 'hist-11-074k-breast.nii.gz-0.500000', 'hist-12-074k-breast.nii.gz-0.500000', 'hist-13-074k-breast.nii.gz-0.500000', 'hist-14-074k-breast.nii.gz-0.500000', 'hist-15-074k-breast.nii.gz-0.500000', 'hist-16-074k-breast.nii.gz-0.500000', 'hist-17-074k-breast.nii.gz-0.500000', 'hist-18-074k-breast.nii.gz-0.5000

200it [00:00, 571.90it/s]


1
1 0.6724189771214784
Objective LinearRegression_Objective:


...

1
1 0.8093700787038851
Objective PLSRegression_Objective:


  7%|██████████████████▍                                                                                                                                                                                                                                                                | 535/8000 [00:11<02:34, 48.26it/s]


iterations: 535
Number of used features: 443
Used features: ['num-074k-breast.nii.gz-0.500000', 'sum-074k-breast.nii.gz-0.500000', 'mean-074k-breast.nii.gz-0.500000', 'std-074k-breast.nii.gz-0.500000', 'skew-074k-breast.nii.gz-0.500000', 'kurt-074k-breast.nii.gz-0.500000', 'hist-0-074k-breast.nii.gz-0.500000', 'hist-1-074k-breast.nii.gz-0.500000', 'hist-2-074k-breast.nii.gz-0.500000', 'hist-3-074k-breast.nii.gz-0.500000', 'hist-4-074k-breast.nii.gz-0.500000', 'hist-5-074k-breast.nii.gz-0.500000', 'hist-6-074k-breast.nii.gz-0.500000', 'hist-7-074k-breast.nii.gz-0.500000', 'hist-8-074k-breast.nii.gz-0.500000', 'hist-9-074k-breast.nii.gz-0.500000', 'hist-10-074k-breast.nii.gz-0.500000', 'hist-11-074k-breast.nii.gz-0.500000', 'hist-12-074k-breast.nii.gz-0.500000', 'hist-13-074k-breast.nii.gz-0.500000', 'hist-14-074k-breast.nii.gz-0.500000', 'hist-15-074k-breast.nii.gz-0.500000', 'hist-16-074k-breast.nii.gz-0.500000', 'hist-17-074k-breast.nii.gz-0.500000', 'hist-18-074k-breast.nii.gz-0.5000

200it [00:00, 267.42it/s]

1
1 0.851005959762946


### breast mean mask

In [18]:
results_no_fs.append(model_selection(breast_features[[c for c in breast_features.columns if 'mean_mask' in c or c == 'type']], breast_target, dataset='breast', type='mean_mask', disable_feature_selection=True))

Objective KNNR_Objective:


  6%|█████████████████▌                                                                                                                                                                                                                                                                 | 511/8000 [00:15<03:43, 33.45it/s]


iterations: 511
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.722505840491539


200it [00:00, 732.03it/s]


1
1 0.7278151564050614
Objective LinearRegression_Objective:


...

1
1 0.891036897419159
Objective PLSRegression_Objective:


  6%|█████████████████                                                                                                                                                                                                                                                                 | 499/8000 [00:00<00:07, 982.45it/s]


iterations: 499
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.881764604721327


200it [00:00, 581.51it/s]

1
1 0.8837435910278285


### breast individual masks

In [19]:
for m in masks:
    results_no_fs.append(model_selection(breast_features[[c for c in breast_features.columns if m in c or c == 'type']], breast_target, dataset='breast', type=m, disable_feature_selection=True))

Objective KNNR_Objective:


  7%|███████████████████▉                                                                                                                                                                                                                                                               | 581/8000 [00:17<03:40, 33.66it/s]


iterations: 581
Number of used features: 26
Used features: ['num-074k-breast.nii.gz-0.500000', 'sum-074k-breast.nii.gz-0.500000', 'mean-074k-breast.nii.gz-0.500000', 'std-074k-breast.nii.gz-0.500000', 'skew-074k-breast.nii.gz-0.500000', 'kurt-074k-breast.nii.gz-0.500000', 'hist-0-074k-breast.nii.gz-0.500000', 'hist-1-074k-breast.nii.gz-0.500000', 'hist-2-074k-breast.nii.gz-0.500000', 'hist-3-074k-breast.nii.gz-0.500000', 'hist-4-074k-breast.nii.gz-0.500000', 'hist-5-074k-breast.nii.gz-0.500000', 'hist-6-074k-breast.nii.gz-0.500000', 'hist-7-074k-breast.nii.gz-0.500000', 'hist-8-074k-breast.nii.gz-0.500000', 'hist-9-074k-breast.nii.gz-0.500000', 'hist-10-074k-breast.nii.gz-0.500000', 'hist-11-074k-breast.nii.gz-0.500000', 'hist-12-074k-breast.nii.gz-0.500000', 'hist-13-074k-breast.nii.gz-0.500000', 'hist-14-074k-breast.nii.gz-0.500000', 'hist-15-074k-breast.nii.gz-0.500000', 'hist-16-074k-breast.nii.gz-0.500000', 'hist-17-074k-breast.nii.gz-0.500000', 'hist-18-074k-breast.nii.gz-0.50000

200it [00:00, 698.44it/s]


1
1 0.6520966568675548
Objective LinearRegression_Objective:


...

1
1 0.865597736054329
Objective PLSRegression_Objective:


  6%|█████████████████▎                                                                                                                                                                                                                                                                | 507/8000 [00:00<00:07, 988.19it/s]


iterations: 507
Number of used features: 26
Used features: ['num-092k-breast.nii.gz-0.500000', 'sum-092k-breast.nii.gz-0.500000', 'mean-092k-breast.nii.gz-0.500000', 'std-092k-breast.nii.gz-0.500000', 'skew-092k-breast.nii.gz-0.500000', 'kurt-092k-breast.nii.gz-0.500000', 'hist-0-092k-breast.nii.gz-0.500000', 'hist-1-092k-breast.nii.gz-0.500000', 'hist-2-092k-breast.nii.gz-0.500000', 'hist-3-092k-breast.nii.gz-0.500000', 'hist-4-092k-breast.nii.gz-0.500000', 'hist-5-092k-breast.nii.gz-0.500000', 'hist-6-092k-breast.nii.gz-0.500000', 'hist-7-092k-breast.nii.gz-0.500000', 'hist-8-092k-breast.nii.gz-0.500000', 'hist-9-092k-breast.nii.gz-0.500000', 'hist-10-092k-breast.nii.gz-0.500000', 'hist-11-092k-breast.nii.gz-0.500000', 'hist-12-092k-breast.nii.gz-0.500000', 'hist-13-092k-breast.nii.gz-0.500000', 'hist-14-092k-breast.nii.gz-0.500000', 'hist-15-092k-breast.nii.gz-0.500000', 'hist-16-092k-breast.nii.gz-0.500000', 'hist-17-092k-breast.nii.gz-0.500000', 'hist-18-092k-breast.nii.gz-0.50000

200it [00:00, 603.11it/s]

1
1 0.850806980622591


### Saving the resulst

In [20]:
results_no_fs= pd.concat(results_no_fs)
results_no_fs.to_csv(os.path.join(path_prefix_results,'results_no_fs.csv'), index=False)
pickle.dump(results_no_fs, open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'wb'))